First download the model weights and parameters, and install caffe

In [1]:
import caffe

In [2]:
caffe.set_mode_cpu()

In [ ]:
#load the model
net = caffe.Net('models/bvlc_reference_caffenet/deploy.prototxt',
                'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)